In [0]:
import pandas as pd

df1 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_2020_Present.csv")
df2 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_2010_2019.csv")
df3 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_2000_2009.csv")
df4 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_1991_1999.csv")
df5 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_2010_Present.csv")
df6 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_1991_2009.csv")

In [0]:
merged_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

merged_df

In [0]:
merged_df.dtypes

In [0]:
#Dropping all categorical columns
merged_df = merged_df.drop(columns=[col for col in merged_df.select_dtypes('object').columns if col != 'LoanStatus'])

merged_df.head()


In [0]:
merged_df.isnull().sum()

In [0]:
merged_df.isnull().sum()

In [0]:
merged_df=merged_df.apply(
    lambda x: x.fillna('Unknown') if x.dtype == 'object' else x.fillna(x.median())
)

In [0]:
merged_df.isnull().sum()

In [0]:
merged_model = merged_df[merged_df['LoanStatus'].isin(['CHGOFF','PIF'])].copy()
merged_model['loan_default'] = merged_model['LoanStatus'].apply(lambda x: 1 if x=='CHGOFF' else 0)

In [0]:
merged_model['loan_default'].value_counts()

In [0]:
import pandas as pd
#Correlation Matrix

corr = merged_model.corr(numeric_only=True)

corr_with_target = corr['loan_default'].sort_values(ascending=False)

print(corr_with_target)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score


In [0]:
top_naics = merged_model['NAICSCode'].value_counts().nlargest(20).index
merged_model['NAICSCode'] = merged_model['NAICSCode'].where(merged_model['NAICSCode'].isin(top_naics), 'Other')

top_districts = merged_model['CongressionalDistrict'].value_counts().nlargest(10).index
merged_model['CongressionalDistrict'] = merged_model['CongressionalDistrict'].where(merged_model['CongressionalDistrict'].isin(top_districts), 'Other')

merged_model_encoded = pd.get_dummies(merged_model, columns=['NAICSCode', 'CongressionalDistrict'], drop_first=True)
merged_model_encoded


In [0]:
merged_model_encoded['loan_default'].value_counts()

In [0]:
from sklearn.utils import resample,shuffle
dl1 = merged_model_encoded[merged_model_encoded['loan_default']==1]

other_df = merged_model_encoded[merged_model_encoded['loan_default']!=1]

dl1_upsampled = resample(dl1, random_state=42, n_samples=1130042, replace=True)

merged_upsampled = pd.concat([dl1_upsampled, other_df])
merged_upsampled['loan_default'].value_counts()

In [0]:
merged_upsampled

In [0]:
base_features = ['TerminMonths', 'ApprovalFY', 'GrossApproval']  

# Select all NAICSCode and CongressionalDistrict dummy columns
naics_cols = merged_upsampled.filter(like='NAICSCode_').columns.tolist()
district_cols = merged_upsampled.filter(like='CongressionalDistrict_').columns.tolist()


features = base_features + naics_cols + district_cols
target = 'loan_default'

In [0]:
from sklearn.model_selection import train_test_split

X = merged_upsampled[features]
y = merged_upsampled[target]

# Split 70% train / 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [0]:
from sklearn.linear_model import LogisticRegression

# Instance of model
model = LogisticRegression(class_weight='balanced', max_iter=1000)  # max_iter ensures convergence

# Fit model on training data
model.fit(X_train, y_train)


In [0]:
model.coef_

In [0]:
y_pred = model.predict(X_test)

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Full classification report (precision, recall, f1-score)
print(classification_report(y_test, y_pred))


In [0]:
from sklearn.metrics import RocCurveDisplay


RocCurveDisplay.from_predictions(y_test, y_pred)


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap")
plt.show()

In [0]:
sns.boxplot(x='loan_default', y='TerminMonths', data=merged_upsampled)
plt.title("Boxplot of TerminMonths by loan_default")
plt.show()

In [0]:
sns.boxplot(x='loan_default', y='ApprovalFY', data=merged_upsampled)
plt.title("Boxplot of TerminMonths by loan_default")
plt.show()

In [0]:
sns.boxplot(x='loan_default', y='GrossApproval', data=merged_upsampled)
plt.title("Boxplot of TerminMonths by loan_default")
plt.show()

In [0]:
merged_upsampled.columns

In [0]:
clean_model = merged_upsampled.drop(['NAICSCode_445310.0', 'NAICSCode_447110.0',
       'NAICSCode_484110.0', 'NAICSCode_484121.0', 'NAICSCode_531311.0',
       'NAICSCode_541110.0', 'NAICSCode_561730.0', 'NAICSCode_621111.0',
       'NAICSCode_621210.0', 'NAICSCode_621310.0', 'NAICSCode_624410.0',
       'NAICSCode_713940.0', 'NAICSCode_721110.0', 'NAICSCode_722110.0',
       'NAICSCode_722211.0', 'NAICSCode_722511.0', 'NAICSCode_722513.0',
       'NAICSCode_811111.0', 'NAICSCode_812112.0', 'NAICSCode_Other',
       'CongressionalDistrict_1.0', 'CongressionalDistrict_2.0',
       'CongressionalDistrict_3.0', 'CongressionalDistrict_4.0',
       'CongressionalDistrict_5.0', 'CongressionalDistrict_6.0',
       'CongressionalDistrict_7.0', 'CongressionalDistrict_8.0',
       'CongressionalDistrict_9.0', 'CongressionalDistrict_Other'], axis=1)

In [0]:
clean_model['loan_default'].value_counts()

In [0]:
merged_model['loan_default'].value_counts()

In [0]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.combine import SMOTETomek

In [0]:
merged_model['loan_to_job_ratio'] = merged_model['GrossApproval'] / merged_model['JobsSupported']

In [0]:
# safer way to fill nulls
merged_model['loan_to_job_ratio'] = merged_model['loan_to_job_ratio'].fillna(0)
merged_model.isnull().sum()

In [0]:
#Create features
features = ['TerminMonths', 'ApprovalFY', 'GrossApproval','loan_to_job_ratio']  
target = 'loan_default'

#test train split
X_train, X_test, y_train, y_test = train_test_split(merged_model[features], merged_model[target], test_size=0.3, random_state=42)

In [0]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  #gets mean and std then transforms by subtracting mean and dividing by std
X_test_scaled = scaler.transform(X_test) #to prevent data leakage use mean and std from training data

smote = SMOTE(sampling_strategy=0.5, random_state=42)
#resampler = SMOTETomek(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train_scaled, y_train)

In [0]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_sm, y_train_sm)

In [0]:
y_pred = logreg.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [0]:
print("Confusion Matrix:\n", ConfusionMatrixDisplay.from_predictions(y_test, y_pred))

In [0]:
logreg.coef_